In [1]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import numpy as np
import re
import os
from datetime import datetime

base_path = '/content/drive/MyDrive/dataset_autoscaling/'
file_jul = os.path.join(base_path, 'access_log_Jul95.txt')
file_aug = os.path.join(base_path, 'access_log_Aug95.txt')

print("Đã thiết lập môi trường xong!")

Mounted at /content/drive
Đã thiết lập môi trường xong!


In [ ]:
# Định nghĩa Regex pattern để bắt các nhóm thông tin trong 1 dòng log
# Mẫu log NASA: host - - [timestamp] "request" status bytes
LOG_PATTERN = re.compile(r'^(\S+) \S+ \S+ \[(.*?)\] "(.*?)" (\d{3}) (\S+)')

def parse_log_file(file_path):
    """
    Hàm đọc file log và chuyển thành DataFrame
    """
    print(f"Đang xử lý file: {file_path}")

    data = []
    errors = 0

    # Mở file với encoding 'latin-1' để tránh lỗi ký tự lạ
    with open(file_path, 'r', encoding='latin-1') as f:
        for line in f:
            match = LOG_PATTERN.match(line)
            if match:
                # Nếu khớp mẫu, tách dữ liệu ra
                host, timestamp, request, status, size = match.groups()

                # Xử lý trường Bytes: nếu là '-' thì coi là 0
                if size == '-':
                    size = 0
                else:
                    size = int(size)

                data.append([host, timestamp, request, int(status), size])
            else:
                errors += 1

    print(f"Số dòng lỗi không đọc được: {errors}")

    # Tạo DataFrame
    columns = ['host', 'timestamp', 'request', 'status', 'bytes']
    df = pd.DataFrame(data, columns=columns)
    return df

In [ ]:
# Đọc file Tháng 7
df_jul = parse_log_file(file_jul)
print(f"Kích thước tháng 7: {df_jul.shape}")

# Đọc file Tháng 8
df_aug = parse_log_file(file_aug)
print(f"Kích thước tháng 8: {df_aug.shape}")

# Gộp 2 tháng lại thành 1 và bỏ qua chỉ số index cũ
df_total = pd.concat([df_jul, df_aug], ignore_index=True)
print(f"Tổng kích thước dữ liệu: {df_total.shape}")

# Xem thử 5 dòng đầu tiên xem đúng format chưa
df_total.head()

Đang xử lý file: /content/drive/MyDrive/dataset_autoscaling/access_log_Jul95.txt
Số dòng lỗi không đọc được: 0
Kích thước tháng 7: (2934961, 5)
Đang xử lý file: /content/drive/MyDrive/dataset_autoscaling/access_log_Aug95.txt
Số dòng lỗi không đọc được: 0
Kích thước tháng 8: (526651, 5)
Tổng kích thước dữ liệu: (3461612, 5)


,host,timestamp,request,status,bytes
0,199.72.81.55,01/Jul/1995:00:00:01 -0400,GET /history/apollo/ HTTP/1.0,200,6245
1,unicomp6.unicomp.net,01/Jul/1995:00:00:06 -0400,GET /shuttle/countdown/ HTTP/1.0,200,3985
2,199.120.110.21,01/Jul/1995:00:00:09 -0400,GET /shuttle/missions/sts-73/mission-sts-73.ht...,200,4085
3,burger.letters.com,01/Jul/1995:00:00:11 -0400,GET /shuttle/countdown/liftoff.html HTTP/1.0,304,0
4,199.120.110.21,01/Jul/1995:00:00:11 -0400,GET /shuttle/missions/sts-73/sts-73-patch-smal...,200,4179


In [ ]:
print("Đang chuyển đổi định dạng thời gian")

# Định dạng thời gian trong log NASA: %d/%b/%Y:%H:%M:%S %z
# Ví dụ: 01/Jul/1995:00:00:01 -0400
df_total['timestamp'] = pd.to_datetime(df_total['timestamp'], format='%d/%b/%Y:%H:%M:%S %z', errors='coerce')

# Set timestamp làm Index (để tiện resample sau này)
df_total.set_index('timestamp', inplace=True)

print("Đã chuyển đổi xong!")
df_total.head()

Đang chuyển đổi định dạng thời gian
Đã chuyển đổi xong!


,host,request,status,bytes
timestamp,,,,
1995-07-01 00:00:01-04:00,199.72.81.55,GET /history/apollo/ HTTP/1.0,200,6245
1995-07-01 00:00:06-04:00,unicomp6.unicomp.net,GET /shuttle/countdown/ HTTP/1.0,200,3985
1995-07-01 00:00:09-04:00,199.120.110.21,GET /shuttle/missions/sts-73/mission-sts-73.ht...,200,4085
1995-07-01 00:00:11-04:00,burger.letters.com,GET /shuttle/countdown/liftoff.html HTTP/1.0,304,0
1995-07-01 00:00:11-04:00,199.120.110.21,GET /shuttle/missions/sts-73/sts-73-patch-smal...,200,4179


In [ ]:
# Lưu file tổng hợp đã xử lý
output_path = os.path.join(base_path, 'nasa_logs_processed.parquet')
df_total.to_parquet(output_path)

print(f"Đã lưu dữ liệu sạch vào: {output_path}")
print("Bạn đã hoàn thành bước Data Ingestion!")

Đã lưu dữ liệu sạch vào: /content/drive/MyDrive/dataset_autoscaling/nasa_logs_processed.parquet
Bạn đã hoàn thành bước Data Ingestion!
